In [6]:
%%sh
export PYTHONPATH=/home/mash-intern/mash/cue:${PYTHONPATH}

In [7]:
!pwd

/home/mash-intern/mash/cue/test


In [1]:
import sys
import os

# Get the directory containing the notebook file
notebook_dir = os.path.dirname(os.path.abspath('__file__'))

# Append the parent directory of the notebook directory to the Python path
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
sys.path.append(parent_dir)

import config_utils as config_utils
import test
import core as core
import argparse
from collections import defaultdict
from torch.utils.data import DataLoader
import torch
import models.cue_net as models
import logging
import seq.io as io
import img.datasets as datasets
from img.refinery import SVKeypointRefinery
import seq.refinery
from img.datasets import SVStreamingDataset
import img.utils as utils
import seq.filters as sv_filters
from seq.aln_index import AlnIndex
from joblib import Parallel, delayed
import seq.utils as seq_utils
import os
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")




In [5]:
data_config = './data/demo/data_config.yaml'
model_config = './data/demo/model_config.yaml'
refine_config = None
skip_inference = False
# load the configs
config = config_utils.load_config(model_config, config_type=config_utils.CONFIG_TYPE.TEST)

[INFO]  ========== Model config ==========
	model_path: ../data/demo/models/cue.pt
	n_cpus: 1
	gpu_ids: []
	batch_size: 16
	logging_level: INFO
	report_interval: 50
	n_jobs_per_gpu: 1
	signal_set: SV_SIGNAL_SET.SHORT
	class_set: SV_CLASS_SET.BASIC5ZYG
	num_keypoints: 1
	model_architecture: HG
	image_dim: 256
	sigma: 10
	stride: 4
	heatmap_peak_threshold: 0.4
	pretrained_refinenn_path: None
	config_file: ./data/demo/model_config.yaml
	experiment_dir: /home/mash-intern/mash/cue/test/data/demo
	devices: [device(type='cpu')]
	device: cpu
	log_dir: /home/mash-intern/mash/cue/test/data/demo/logs/
	report_dir: /home/mash-intern/mash/cue/test/data/demo/reports/
	log_file: /home/mash-intern/mash/cue/test/data/demo/logs/main.log
	classes: ['NEG', 'DEL-HOM', 'INV-HOM', 'DUP-HOM', 'DEL-HET', 'INV-HET', 'DUP-HET', 'IDUP-HOM', 'IDUP-HET']
	num_classes: 9
	n_signals: 6


In [6]:
data_config = config_utils.load_config(data_config, config_type=config_utils.CONFIG_TYPE.DATA)


[INFO] ========== Data config =========
	bam: ../data/demo/inputs/chr21.small.bam
	fai: ../data/demo/inputs/chr21.small.fa.fai
	chr_names: ['chr21']
	log_level: Info
	n_cpus: 1
	logging_level: INFO
	min_refine_buffer: 2000
	refine_buffer_frac_size: 5
	refine_pair_dist_frac_size: 2
	refine_bp_kernels: [0, 50, 500]
	refine_min_support: 2
	refine_disable: False
	min_pair_support: 2
	min_pair_distance: 4000
	max_pair_distance: 1000000
	scan_target_intervals: True
	stream: True
	view_mode: False
	store_img: False
	empty_annotation: False
	bins_per_block: 8000
	min_sv_len: 4000
	min_qual_score: 50
	bam_type: BAM_TYPE.SHORT
	signal_set: SV_SIGNAL_SET.SHORT
	signal_set_origin: SHORT
	bed: None
	blacklist_bed: None
	signal_vmax: {'RD': 600, 'RD_LOW': 800, 'RD_CLIPPED': 600, 'SM': 200, 'SR_RP': 600, 'LR': 600, 'LLRR': 100, 'RL': 100, 'LLRR_VS_LR': 1}
	signal_mapq: {'RD': 20, 'RD_LOW': 0, 'RD_CLIPPED': 20, 'SM': 20, 'SR_RP': 0, 'LR': 0, 'LLRR': 1, 'RL': 1, 'LLRR_VS_LR': 1}
	bin_size: 750
	interva

In [7]:
model = models.MultiSVHG(config)
device = torch.device(config.device)
model.load_state_dict(torch.load(config.model_path, device))
model.to(device)


MultiSVHG(
  (backbone): HourglassBackbone(
    (layers): Sequential(
      (0): Conv(
        (layers): Sequential(
          (0): Conv2d(6, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
      )
      (1): Residual(
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
        (skip): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
        (relu): ReLU()
      )
      (2): MaxPool2d(kernel_size=2

In [22]:
# save model pytorch
torch.save(model, 'cue.pth')
print(type(model))

<class 'models.cue_net.MultiSVHG'>


In [23]:
import torch
model2 = torch.load('/home/mash-intern/mash/cue/test/data/demo/models/cue.pth')
model2.eval()
model2.to(device)
print(type(model2))

# model2

<class 'models.cue_net.MultiSVHG'>


In [5]:
%%sh
python call.py --data_config data/demo/data_config.yaml --model_config data/demo/model_config.yaml

Traceback (most recent call last):
  File "call.py", line 75, in <module>
    config = config_utils.load_config(args.model_config, config_type=config_utils.CONFIG_TYPE.TEST)
  File "/home/mash-intern/mash/cue/test/config_utils.py", line 248, in load_config
    with open(fname) as file:
FileNotFoundError: [Errno 2] No such file or directory: 'model/demo/model_config.yaml'


CalledProcessError: Command 'b'python call.py --data_config data/demo/data_config.yaml --model_config model/demo/model_config.yaml\n'' returned non-zero exit status 1.

In [37]:
%%sh
streamlit run /home/mash-intern/anaconda3/envs/cue/lib/python3.7/site-packages/ipykernel_launcher.py

Process is interrupted.
